# Predicting movie ratings

One of the most common uses of big data is to predict what users want. This allows Google to show you relevant ads, Amazon to recommend relevant products, and Netflix to recommend movies that you might like. This lab will demonstrate how we can use Apache Spark to recommend movies to a user. We will start with some basic techniques, and then use the mllib library's Alternating Least Squares method to make more sophisticated predictions.


## 1. Data Setup

Before starting with the recommendation systems, we need to download the dataset and we need to do a little bit of pre-processing. 

### 1.1 Download
Let's begin with downloading the dataset. If you have already a copy of the dataset you can skip this part. For this lab, we will use [movielens 25M stable benchmark rating dataset](https://files.grouplens.org/datasets/movielens/ml-25m.zip). 




In [2]:
# let's start by downloading the dataset.
import wget
wget.download(url = "https://files.grouplens.org/datasets/movielens/ml-25m.zip", out = "dataset.zip")

100% [......................................................................] 261978986 / 261978986

'dataset.zip'

In [3]:
# let's unzip the dataset
import zipfile
with zipfile.ZipFile("dataset.zip", "r") as zfile:
    zfile.extractall()

### 1.2 Dataset Format

The following table highlights some data from `ratings.csv` (with comma-separated elements):

| UserID | MovieID | Rating | Timestamp  |
|--------|---------|--------|------------|
|...|...|...|...|
|3022|152836|5.0|1461788770|
|3023|169|5.0|1302559971|
|3023|262|5.0|1302559918|
|...|...|...|...|

The following table highlights some data from `movies.csv` (with comma-separated elements):

| MovieID | Title   | Genres | 
|---------|---------|--------|
|...|...|...|
| 209133  |The Riot and the Dance (2018) | (no genres listed) |
| 209135  |Jane B. by Agnès V. (1988) | Documentary\|Fantasy |
|...|...|...|

The `Genres` field has the format

`Genres1|Genres2|Genres3|...` or `(no generes listed)`

The format of these files is uniform and simple, so we can easily parse them using python:
- For each line in the rating dataset, we create a tuple of (UserID, MovieID, Rating). We drop
the timestamp because we do not need it for this exercise.
- For each line in the movies dataset, we create a tuple of (MovieID, Title). We drop the Genres
because we do not need them for this exercise.

### 1.3 Preprocessing

We can begin to preprocess our data. This step includes: 
1) We should drop the timestamp, we do not need it.
2) We should drop the genres, we do not need them.
3) We should parse data according to their intended type. For example, the elements of rating should be floats.
4) Each line should encode data in an easily processable format, like a tuple. 
5) We should filter the first line of both datasets (the header).

In [4]:
# let's intialize the spark session

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
                    .appName("Python Spark SQL basic example") \
                    .getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/22 09:40:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### 1.3.1 Load The Data

We can start by loading the dataset formatted as raw text. 

In [5]:
from pprint import pprint
ratings_rdd = spark.sparkContext.textFile(name = "ml-25m/ratings.csv", minPartitions = 2)
movies_rdd  = spark.sparkContext.textFile(name = "ml-25m/movies.csv" , minPartitions = 2)

# let's have a peek a our dataset
print("ratings --->")
pprint(ratings_rdd.take(5))

print("\nmovies --->")
pprint(movies_rdd.take(5))

ratings --->


['userId,movieId,rating,timestamp',
 '1,296,5.0,1147880044',
 '1,306,3.5,1147868817',
 '1,307,5.0,1147868828',
 '1,665,5.0,1147878820']

movies --->
['movieId,title,genres',
 '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 '2,Jumanji (1995),Adventure|Children|Fantasy',
 '3,Grumpier Old Men (1995),Comedy|Romance',
 '4,Waiting to Exhale (1995),Comedy|Drama|Romance']


#### 1.3.2 SubSampling
Since we have limited resources in terms of computation, sometimes, it is useful to work with only a fraction of the whole dataset.

In [6]:
ratings_rdd = ratings_rdd.sample(withReplacement=False, fraction=1/25, seed=14).cache()
movies_rdd  = movies_rdd .sample(withReplacement=False, fraction=1, seed=14).cache()

print(f"ratings_rdd: {ratings_rdd.count()}, movies_rdd {movies_rdd.count()}")

ratings_rdd: 998879, movies_rdd 62424


#### 1.3.2 Parsing
Here, we do the real preprocessing: dropping columns, parsing elements, and filtering the heading.

In [7]:
def string2rating(line):
    """ Parse a line in the ratings dataset.
    Args:
        line (str): a line in the ratings dataset in the form of UserID,MovieID,Rating,Timestamp
    Returns:
        tuple[int,int,float]: (UserID, MovieID, Rating)
    """
    userID, movieID, rating, *others = line.split(",")
    try: return int(userID), int(movieID), float(rating),
    except ValueError: return None

def string2movie(line):
    """ Parse a line in the movies dataset.
    Args:
        line (str): a line in the movies dataset in the form of MovieID,Title,Genres. 
                    Genres in the form of Genre1|Genre2|...
    Returns:
        tuple[int,str,list[str]]: (MovieID, Title, Genres)
    """
    movieID, title, *others = line.split(",")
    try: return int(movieID), title
    except ValueError: return None

ratings_rdd = ratings_rdd.map(string2rating).filter(lambda x:x!=None).cache()
movies_rdd  = movies_rdd .map(string2movie ).filter(lambda x:x!=None).cache()

In [8]:
print(f"There are {ratings_rdd.count()} ratings and {movies_rdd.count()} movies in the datasets")
print(f"Ratings: ---> \n{ratings_rdd.take(3)}")
print(f"Movies: ---> \n{movies_rdd.take(3)}")

There are 998879 ratings and 62423 movies in the datasets
Ratings: ---> 
[(1, 1653, 4.0), (1, 2068, 2.5), (1, 8973, 4.0)]
Movies: ---> 
[(1, 'Toy Story (1995)'), (2, 'Jumanji (1995)'), (3, 'Grumpier Old Men (1995)')]


## 2. Basic Raccomandations


### 2.1 Highest Average Rating.

One way to recommend movies is to always recommend the movies with the highest average rating. In this section, we will use Spark to find the name, number of ratings, and the average rating of the 20 movies with the highest average rating and more than 500 reviews. We want to filter our movies with high ratings but fewer than or equal to 500 reviews because movies with few reviews may not have broad appeal to everyone.

In [9]:
def averageRating(ratings):
    """ Computes the average rating.
        Args:
            tuple[int, list[float]]: a MovieID with its list of ratings
        Returns:
            tuple[int, float]: returns the the MovieID with its average rating.
    """ 
    return (ratings[0], sum(ratings[1]) / len(ratings[1]))

rdd = ratings_rdd.map(lambda x:(x[0], x[2])).groupByKey() # group by MovieID
rdd = rdd.filter(lambda x:len(x[1])>500)                  # filter movies with less than 500 reviews 
rdd = rdd.map(averageRating)                              # computes the average Rating
rdd = rdd.sortBy(lambda x:x[1], ascending=False)

rdd.take(5)

[(72315, 3.050965250965251)]

Ok, now we have the best (according to the average) popular (according to the number of reviews) movies. However, we can only see their MovieID. Let's convert the IDs into titles.

In [10]:
rdd.join(movies_rdd)\
   .map(lambda x:(x[1][1],x[1][0]))\
   .sortBy(lambda x:x[1], ascending=False)\
   .take(20)

[("Hell's Hinges (1916)", 3.050965250965251)]

### 2.2 Collaborative Filtering

We are going to use a technique called collaborative filtering. Collaborative filtering is a method of making automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion as a person B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a person chosen randomly.

At first, people rate different items (like videos, images, games). After that, the system is making predictions about a user's rating for an item, which the user has not rated yet. These predictions are built upon the existing ratings of other users, who have similar ratings with the active user.

#### 2.2.1 Creating a Training Set

Before we jump into using machine learning, we need to break up the `ratings_rdd` dataset into three pieces:

* a training set (RDD), which we will use to train models,
* a validation set (RDD), which we will use to choose the best model,
* a test set (RDD), which we will use for estimating the predictive power of the recommender system.

To randomly split the dataset into multiple groups, we can use the pyspark [randomSplit] transformation, which takes a list of splits with a seed and returns multiple RDDs.

[randomSplit]:https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.randomSplit.html?highlight=randomsplit#pyspark.RDD.randomSplit

In [11]:
training_rdd, validation_rdd, test_rdd = ratings_rdd.randomSplit([6, 2, 2], seed=14)

print(f"Training: {training_rdd.count()}, validation: {validation_rdd.count()}, test: {test_rdd      .count()}")

print("training   samples: ", training_rdd  .take(3))
print("validation samples: ", validation_rdd.take(3))
print("test       samples: ", test_rdd      .take(3))

Training: 599932, validation: 199477, test: 199470
training   samples:  [(1, 1653, 4.0), (1, 8973, 4.0), (2, 261, 0.5)]
validation samples:  [(2, 1257, 5.0), (2, 2745, 5.0), (3, 61132, 3.0)]
test       samples:  [(1, 2068, 2.5), (2, 260, 5.0), (2, 457, 5.0)]


#### 2.2.2 Alternating Least Square Errors

For movie recommendations, we start with a matrix whose entries are movie ratings by users.  Each column represents a user and each row represents a particular movie.

Since not all users have rated all movies, we do not know all of the entries in this matrix, which is precisely why we need collaborative filtering.  For each user, we have ratings for only a subset of the movies.  With collaborative filtering, the idea is to approximate the rating matrix by factorizing it as the product of two matrices: one that describes properties of each user, and one that describes properties of each movie.

We want to select these two matrices such that the error for the users/movie pairs where we know the correct ratings is minimized.  The *Alternating Least Squares* algorithm does this by first randomly filling the user matrix with values and then optimizing the value of the movies such that the error is minimized.  Then, it holds the movies matrix constant and optimizes the value of the user's matrix.  This alternation between which matrix to optimize is the reason for the "alternating" in the name.


In [12]:
from pyspark.mllib.recommendation import ALS

# thanks to modern libraries training an ALS model is as easy as
model = ALS.train(training_rdd, rank = 4, seed = 14, iterations = 5, lambda_ = 0.1)

# let's have a peek to few predictions
model.predictAll(validation_rdd.map(lambda x:(x[0],x[1]))).take(5)

22/02/22 09:41:26 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/02/22 09:41:26 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/02/22 09:41:27 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


[Rating(user=141470, product=1732, rating=3.1807164083245922),
 Rating(user=18500, product=5621, rating=2.4928555764767624),
 Rating(user=18500, product=8533, rating=3.1193395535588895),
 Rating(user=18500, product=6763, rating=2.2404433916172453),
 Rating(user=18500, product=80363, rating=2.369844745977925)]

#### 2.2.3 Root Mean Square Error (RMSE)

Next, we need to evaluate our model: is it good or is it bad?

To score the model, we will use RMSE (often called also Root Mean Square Deviation (RMSD)). You can think of RMSE as a distance function that measures the distance between the predictions and the ground truths. It is computed as follows: 

$$ RMSE(f, \mathcal{D}) = \sqrt{\frac{\sum_{(x,y) \in \mathcal{D}} (f(x) - y)^2}{|\mathcal{D}|}}$$

Where:
* $\mathcal{D}$ is our dataset it contains samples alongside their predictions. Formally, $\mathcal{D} \subseteq \mathcal{X} \times \mathcal{Y}$. Where:
    * $\mathcal{X}$ is the set of all input samples.
    * $\mathcal{Y}$ is the set of all possible predictions. 
* $f : \mathcal{X} \rightarrow \mathcal{Y}$ is the model we wish to evaluate. Given an input $x$ (from $\mathcal{X}$, the set of possible inputs) it returns a value $f(x)$ (from $\mathcal{Y}$, the set of possible outputs). 
* $x$ represents an input. 
* $f(x)$ represents the prediction of $x$.
* $y$ represents the ground truth.

As you can imagine $f(x)$ and $y$ can be different, i.e our model is wrong. With $RMSE(f, \mathcal{D})$, we want to measure the degree to which our model, $f$, is wrong on the dataset $\mathcal{D}$. The higher is $RMSE(f, \mathcal{D})$ the higher is the degree to which $f$ is wrong. The smaller is $RMSE(f, \mathcal{D})$ the more accurate $f$ is. 

To better understand the RMSE consider the following facts:
* When $f(x)$ is close to $y$ our model is accurate. In the same case $(f(x) - y)^2$ is small.
* When $f(x)$ is far from $y$ our model is inaccurate. In the same case $(f(x) - y)^2$ is high.
* If our model is accurate, it will be often accurate in $\mathcal{D}$. Therefore, it will make often small errors which will amount to a small RMSE. 
* If our model is inaccurate, it will be often inaccurate in $\mathcal{D}$. Therefore, it will make often big errors which will amount to a large RMSE.

Let's make a function to compute the RMSE so that we can use it multiple times easily.

In [13]:

def RMSE(predictions_rdd, truths_rdd):
    """ Compute the root mean squared error between predicted and actual
    Args:
        predictions_rdd: predicted ratings for each movie and each user where each entry is in the form (UserID, MovieID, Rating).
        truths_rdd: actual ratings where each entry is in the form (UserID, MovieID, Rating).
    Returns:
        RSME (float): computed RSME value
    """
    # Transform predictions and truths into the tuples of the form ((UserID, MovieID), Rating)
    predictions = predictions_rdd.map(lambda i: ((i[0], i[1]), i[2]))
    truths      = truths_rdd     .map(lambda i: ((i[0], i[1]), i[2]))

    # Compute the squared error for each matching entry (i.e., the same (User ID, Movie ID) in each
    # RDD) in the reformatted RDDs using RDD transformtions - do not use collect()
    squared_errors = predictions.join(truths)\
                                .map(lambda i: (i[1][0] - i[1][1])**2)
                       

    total_squared_error     = squared_errors.sum()
    total_ratings           = squared_errors.count()
    mean_squared_error      = total_squared_error / total_ratings
    root_mean_squared_error = mean_squared_error ** (1/2)
    
    return root_mean_squared_error


In [14]:
# let's evaluate the trained models

RMSE(predictions_rdd = model.predictAll(validation_rdd.map(lambda x:(x[0],x[1]))),
     truths_rdd      = validation_rdd)


1.1882250104369456

#### 2.2.4 HyperParameters Tuning

Can we do better? 

When training the ALS model there were few parameters to set. However, we do not know which is the best configuration. On these occasions, we want to try a few combinations to obtain even better results. In this section, we will search a few parameters. We will perform a so-called **grid search**. We will proceed as follows:

1) We decide the parameters to tune.
2) We train with all possible configurations. 
3) We evaluate a trained model with all possible configurations on the validation set.
4) We evaluate the best model on the test set.

In [15]:

HyperParameters = {
    "rank"       : [4, 8, 12],
    "seed"       : [14],
    "iterations" : [5, 10], 
    "lambda"     : [0.05, 0.1, 0.25]
}

best_model = None
best_error = float("inf")
best_conf  = dict()

# how many training are we doing ?
for rank in HyperParameters["rank"]:                     #
    for seed in HyperParameters["seed"]:                 # I consider these nested for-loops an anti-pattern.
        for iterations in HyperParameters["iterations"]: # However, We can leave as it is for sake of simplicity. 
            for lambda_ in HyperParameters["lambda"]:    # 
                
                model = ALS.train(training_rdd, rank = rank, seed = seed, iterations = iterations, lambda_ = lambda_)
                validation_error = RMSE(predictions_rdd = model.predictAll(validation_rdd.map(lambda x:(x[0],x[1]))),
                                        truths_rdd      = validation_rdd)
                
                if validation_error < best_error: 
                    best_model, best_error = model, validation_error
                    best_conf = {"rank":rank, "seed":seed, "iterations":iterations, "lambda":lambda_}
                    print(f"current best validation error {best_error} with configuration {best_conf}")

test_error = RMSE(predictions_rdd = model.predictAll(test_rdd.map(lambda x:(x[0],x[1]))), truths_rdd = test_rdd)
print(f"test error {test_error} with configuration {best_conf}")

current best validation error 1.3506269717633657 with configuration {'rank': 4, 'seed': 14, 'iterations': 5, 'lambda': 0.05}


current best validation error 1.1882250104369456 with configuration {'rank': 4, 'seed': 14, 'iterations': 5, 'lambda': 0.1}


current best validation error 1.0803793995678013 with configuration {'rank': 4, 'seed': 14, 'iterations': 5, 'lambda': 0.25}


current best validation error 1.0295688329770294 with configuration {'rank': 4, 'seed': 14, 'iterations': 10, 'lambda': 0.25}


current best validation error 1.0281684455014197 with configuration {'rank': 8, 'seed': 14, 'iterations': 10, 'lambda': 0.25}


test error 1.0285494437483655 with configuration {'rank': 8, 'seed': 14, 'iterations': 10, 'lambda': 0.25}
